# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [1]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql import Window
from pyspark.sql.functions import isnan, count, when, col, desc, udf, col, sort_array, asc, avg, sum, max
from pyspark.ml.feature import VectorAssembler, Normalizer, StandardScaler
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import RandomForestClassifier, LinearSVC, LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt


In [2]:
# Create a Spark session
spark = SparkSession \
    .builder \
    .appName("Spark Project") \
    .getOrCreate()

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [3]:
# Set path and load dataset
path = "mini_sparkify_event_data.json"
user_log = spark.read.json(path)

In [4]:
# Check first few entries
#user_log.take(3)

In [5]:
# Drop entries missing values for userID or sessionId
user_log_valid = user_log.dropna(how = "any", subset = ["userId", "sessionId"])

user_log_valid = user_log_valid.filter(user_log_valid["userId"] != "")


In [6]:
# Find the total number of log entries
user_log_valid.count()

278154

In [7]:
# What are the existing data fields
# Check Schema
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [8]:
# Show all possible results for page and check for blanks
user_log_valid.select("page").dropDuplicates().show()

+--------------------+
|                page|
+--------------------+
|              Cancel|
|    Submit Downgrade|
|         Thumbs Down|
|                Home|
|           Downgrade|
|         Roll Advert|
|              Logout|
|       Save Settings|
|Cancellation Conf...|
|               About|
|            Settings|
|     Add to Playlist|
|          Add Friend|
|            NextSong|
|           Thumbs Up|
|                Help|
|             Upgrade|
|               Error|
|      Submit Upgrade|
+--------------------+



# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [9]:
#First look at what data looks like
# Check first few entries
user_log_valid.take(3)

[Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30'),
 Row(artist='Five Iron Frenzy', auth='Logged In', firstName='Micah', gender='M', itemInSession=79, lastName='Long', length=236.09424, level='free', location='Boston-Cambridge-Newton, MA-NH', method='PUT', page='NextSong', registration=1538331630000, sessionId=8, song='Canada', status=200, ts=1538352180000, userAgent='"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.103 Safari/537.36"', userId='9'),
 Row(artist='Adam Lambert', auth='Logged In', firstName='Colin', gender='M', itemInSession=51, lastName='Freeman', length=282.8273, level='paid', location='

In [10]:
# Total number of entries
user_log_valid.count()

278154

In [11]:
# What are the existing data fields
# Check Schema
user_log_valid.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [12]:
# Look at example series of entries for one user
user_log_valid.select("ts","page").where(user_log_valid.userId == 30).show(10)

+-------------+---------------+
|           ts|           page|
+-------------+---------------+
|1538352117000|       NextSong|
|1538352394000|       NextSong|
|1538352676000|       NextSong|
|1538352899000|       NextSong|
|1538352905000|Add to Playlist|
|1538353084000|       NextSong|
|1538353218000|       NextSong|
|1538353441000|       NextSong|
|1538353687000|       NextSong|
|1538353909000|       NextSong|
+-------------+---------------+
only showing top 10 rows



In [13]:
# Look at page possibilities
user_log_valid.select("page").dropDuplicates().show()

+--------------------+
|                page|
+--------------------+
|              Cancel|
|    Submit Downgrade|
|         Thumbs Down|
|                Home|
|           Downgrade|
|         Roll Advert|
|              Logout|
|       Save Settings|
|Cancellation Conf...|
|               About|
|            Settings|
|     Add to Playlist|
|          Add Friend|
|            NextSong|
|           Thumbs Up|
|                Help|
|             Upgrade|
|               Error|
|      Submit Upgrade|
+--------------------+



In [14]:
# Look at song info
# Show details of individual fields
user_log_valid.select("level").dropDuplicates().show()

+-----+
|level|
+-----+
| free|
| paid|
+-----+



In [15]:
# Make function for flagging downgrade/delete account events
flag_downgrade_event = udf(lambda x: 1 if (x == "Submit Downgrade" or x == "Cancel") else 0, IntegerType())

In [16]:
# Add downgrade feature to user log
user_log_valid = user_log_valid.withColumn("downgraded", flag_downgrade_event("page"))

In [17]:
# Partition the data set by user
windowval = Window.partitionBy("userId").orderBy(desc("ts")).rangeBetween(Window.unboundedPreceding, 0)

# Add Phase feature to user log
user_log_valid = user_log_valid.withColumn("phase", sum("downgraded").over(windowval))


In [18]:
# Example Results of windowing
user_log_valid.select(["userId", "sessionId", "page", "level", "phase", "downgraded"]).where(user_log.userId == "30").sort("sessionId").collect()

[Row(userId='30', sessionId=29, page='Add to Playlist', level='paid', phase=1, downgraded=0),
 Row(userId='30', sessionId=29, page='NextSong', level='paid', phase=1, downgraded=0),
 Row(userId='30', sessionId=29, page='NextSong', level='paid', phase=1, downgraded=0),
 Row(userId='30', sessionId=29, page='NextSong', level='paid', phase=1, downgraded=0),
 Row(userId='30', sessionId=29, page='NextSong', level='paid', phase=1, downgraded=0),
 Row(userId='30', sessionId=29, page='Roll Advert', level='paid', phase=1, downgraded=0),
 Row(userId='30', sessionId=29, page='NextSong', level='paid', phase=1, downgraded=0),
 Row(userId='30', sessionId=29, page='NextSong', level='paid', phase=1, downgraded=0),
 Row(userId='30', sessionId=29, page='NextSong', level='paid', phase=1, downgraded=0),
 Row(userId='30', sessionId=29, page='NextSong', level='paid', phase=1, downgraded=0),
 Row(userId='30', sessionId=29, page='NextSong', level='paid', phase=1, downgraded=0),
 Row(userId='30', sessionId=29, p

In [19]:
# Find how many accounts were downgraded in the data set
user_log_valid.select("downgraded").where(user_log_valid.downgraded==1).count()

115

In [20]:
# Find total number of user/phase combinations
user_log_valid.select("userId","phase").dropDuplicates().count()

339

In [21]:
# Find total number of sessions
user_log_valid.select("sessionId").dropDuplicates().count()

2312

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

In [22]:
# Add feature for Gender
# Function to flag gender
flag_male = udf(lambda x: 1 if x == "M" else 0, IntegerType())
flag_female = udf(lambda x: 1 if x == "F" else 0, IntegerType())

# Add Gender feature to user log
user_log_valid = user_log_valid.withColumn("Male", flag_male("gender"))
user_log_valid = user_log_valid.withColumn("Female", flag_female("gender"))

In [23]:
# Add feature for subscription level
# Function to free or paid levels
flag_paid = udf(lambda x: 1 if x == "paid" else 0, IntegerType())
flag_free = udf(lambda x: 1 if x == "free" else 0, IntegerType())

# Add Gender feature to user log
user_log_valid = user_log_valid.withColumn("Paid", flag_paid("level"))
user_log_valid = user_log_valid.withColumn("Free", flag_free("level"))

In [24]:
user_log_valid.head()

Row(artist=None, auth='Logged In', firstName='Darianna', gender='F', itemInSession=34, lastName='Carpenter', length=None, level='free', location='Bridgeport-Stamford-Norwalk, CT', method='PUT', page='Logout', registration=1538016340000, sessionId=187, song=None, status=307, ts=1542823952000, userAgent='"Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like Mac OS X) AppleWebKit/537.51.2 (KHTML, like Gecko) Version/7.0 Mobile/11D257 Safari/9537.53"', userId='100010', downgraded=0, phase=0, Male=0, Female=1, Paid=0, Free=1)

In [25]:
# Make functions for flagging key page actions
flag_thumbsdown_event = udf(lambda x: 1 if x == "Thumbs Down" else 0, IntegerType())
flag_thumbsup_event = udf(lambda x: 1 if x == "Thumbs Up" else 0, IntegerType())
flag_rolladvert_event = udf(lambda x: 1 if x == "Roll Advert" else 0, IntegerType())
flag_settings_event = udf(lambda x: 1 if x == "Settings" else 0, IntegerType())
flag_addtoplaylist_event = udf(lambda x: 1 if x == "Add to Playlist" else 0, IntegerType())
flag_addfriend_event = udf(lambda x: 1 if x == "Add Friend" else 0, IntegerType())
flag_help_event = udf(lambda x: 1 if x == "Help" else 0, IntegerType())
flag_error_event = udf(lambda x: 1 if x == "Error" else 0, IntegerType())
flag_song_event = udf(lambda x: 1 if x == "NextSong" else 0, IntegerType())

In [26]:
# Add page event flags to the user log
user_log_valid = user_log_valid.withColumn("thumbsdown", flag_thumbsdown_event("page"))
user_log_valid = user_log_valid.withColumn("thumbsup", flag_thumbsup_event("page"))
user_log_valid = user_log_valid.withColumn("advert", flag_rolladvert_event("page"))
user_log_valid = user_log_valid.withColumn("settings", flag_settings_event("page"))
user_log_valid = user_log_valid.withColumn("addtoplaylist", flag_addtoplaylist_event("page"))
user_log_valid = user_log_valid.withColumn("friend", flag_addfriend_event("page"))
user_log_valid = user_log_valid.withColumn("help", flag_help_event("page"))
user_log_valid = user_log_valid.withColumn("error", flag_error_event("page"))
user_log_valid = user_log_valid.withColumn("song", flag_error_event("page"))

In [27]:
user_log_valid.head()

Row(artist=None, auth='Logged In', firstName='Darianna', gender='F', itemInSession=34, lastName='Carpenter', length=None, level='free', location='Bridgeport-Stamford-Norwalk, CT', method='PUT', page='Logout', registration=1538016340000, sessionId=187, song=0, status=307, ts=1542823952000, userAgent='"Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like Mac OS X) AppleWebKit/537.51.2 (KHTML, like Gecko) Version/7.0 Mobile/11D257 Safari/9537.53"', userId='100010', downgraded=0, phase=0, Male=0, Female=1, Paid=0, Free=1, thumbsdown=0, thumbsup=0, advert=0, settings=0, addtoplaylist=0, friend=0, help=0, error=0)

In [28]:
# First aggregation to find numbers of key page events using flags for each session
agg1 = user_log_valid.groupBy('userId','sessionId').agg({'thumbsdown':'sum', 'thumbsup':'sum', 'advert':'sum', 
                                                     'settings':'sum', 'addtoplaylist':'sum', 'friend':'sum',
                                                     'help':'sum', 'error':'sum', 'downgraded':'max', 'song':'sum',
                                                     'Male':'max','Female':'max','Paid':'max','Free':'max',
                                                     'phase':'max'})

In [29]:
agg1.select('userId','sessionId','sum(thumbsdown)','max(Paid)','max(phase)', 'max(downgraded)').show()

+------+---------+---------------+---------+----------+---------------+
|userId|sessionId|sum(thumbsdown)|max(Paid)|max(phase)|max(downgraded)|
+------+---------+---------------+---------+----------+---------------+
|   101|      635|              8|        1|         1|              0|
|    29|     1030|              0|        1|         1|              0|
|    42|      433|              0|        0|         0|              0|
|     8|     1200|              1|        0|         0|              0|
|    35|     2270|              0|        0|         0|              0|
|   110|     1776|              0|        0|         0|              0|
|    96|     1653|              3|        1|         1|              0|
|   140|      798|              0|        0|         2|              0|
|   120|      627|              2|        1|         0|              0|
|300011|       60|              0|        1|         1|              0|
|    92|      358|              0|        0|         1|         

In [30]:
agg2 = agg1.groupBy('userId','max(phase)').agg({'sum(thumbsdown)':'avg', 'sum(thumbsup)':'avg', 'sum(advert)':'avg', 
                                                'sum(settings)':'avg', 'sum(addtoplaylist)':'avg', 'sum(friend)':'avg',
                                                'sum(help)':'avg', 'sum(error)':'avg', 'sum(song)':'avg', 'max(Male)':'avg',
                                                'max(Female)':'avg','max(Paid)':'avg','max(Free)':'avg','max(downgraded)':'max'})


In [31]:
# Assemble vector with numerical features
assembler = VectorAssembler(inputCols=["avg(sum(thumbsdown))", "avg(sum(thumbsup))", "avg(sum(advert))", "avg(sum(settings))", 
                                       "avg(sum(addtoplaylist))","avg(sum(friend))", "avg(sum(help))", "avg(sum(error))", 
                                       "avg(sum(song))", "avg(max(Male))", "avg(max(Female))", "avg(max(Paid))", 
                                       "avg(max(Free))"], outputCol="NumFeatures")
ML_df = assembler.transform(agg2)

In [32]:
# Normalize features to ensure they are same scale (use mean/standard deviation)
#scaler2 = StandardScaler(inputCol="NumFeatures", outputCol="ScaledFeatures",  withMean=True, withStd=True)
#scalerModel = scaler2.fit(ML_df)
#ML_df = scalerModel.transform(ML_df)

In [33]:
# Use a standard scaler to normalize the feature data for the maximum values in the data set. 
# This ensures that feature values will stay positive, making them easier to understand.
scaler = Normalizer(inputCol="NumFeatures", outputCol="ScaledFeatures")
ML_df = scaler.transform(ML_df)

In [34]:
# Check the results of normalization/feature creation
ML_df.head(5)

[Row(userId='149', max(phase)=0, avg(sum(thumbsup))=3.6666666666666665, avg(sum(help))=0.3333333333333333, avg(max(Female))=0.0, avg(max(Free))=1.0, avg(sum(thumbsdown))=0.3333333333333333, avg(sum(friend))=3.0, avg(sum(song))=0.0, avg(sum(advert))=4.333333333333333, avg(max(Male))=1.0, avg(sum(settings))=1.0, avg(sum(error))=0.0, avg(sum(addtoplaylist))=1.6666666666666667, max(max(downgraded))=0, avg(max(Paid))=0.0, NumFeatures=DenseVector([0.3333, 3.6667, 4.3333, 1.0, 1.6667, 3.0, 0.3333, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0]), ScaledFeatures=DenseVector([0.0485, 0.5336, 0.6306, 0.1455, 0.2425, 0.4366, 0.0485, 0.0, 0.0, 0.1455, 0.0, 0.0, 0.1455])),
 Row(userId='55', max(phase)=0, avg(sum(thumbsup))=1.5454545454545454, avg(sum(help))=0.36363636363636365, avg(max(Female))=0.0, avg(max(Free))=0.9090909090909091, avg(sum(thumbsdown))=0.45454545454545453, avg(sum(friend))=0.7272727272727273, avg(sum(song))=0.09090909090909091, avg(sum(advert))=2.6363636363636362, avg(max(Male))=1.0, avg(sum(setti

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

In [35]:
# Create dataframe with correct labels for easy model fitting
data = ML_df.select(col('max(max(downgraded))').alias("label"), col('ScaledFeatures').alias("features"))

In [36]:
# Split training/rest of the data
train, test = data.randomSplit([0.7, 0.3])
# Split non-training data between test/validation
#test, validation = rest.randomSplit([0.5, 0.5])

In [50]:
# Set up Model
lr = LinearRegression(maxIter=10, regParam=0.0, fitIntercept=False, solver="normal")

In [51]:
# Fit model
lrModel = lr.fit(train)

In [52]:
# See influence of different variables
lrModel.coefficients


DenseVector([1.0933, 0.168, 0.4614, 0.3248, 0.1184, -0.4869, 0.6676, 2.4972, -3.2602, -5.4177, -5.4859, 5.2043, 4.8933])

In [53]:
# See model R2
lrModelSummary = lrModel.summary
lrModelSummary.r2


0.5416801555580528

In [41]:
# Make predictions
results = lrModel.transform(test)

In [54]:
# Set up a Random Forest Classifier Model
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=10)

In [55]:
# Fit the model to the training data
rfModel = rf.fit(train)

In [56]:
# Create predictions for the test data set
predictions = rfModel.transform(test)

In [57]:
# Evaluate the F1 score for the basic Random Forest Classifier
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
F1 = evaluator.evaluate(predictions)
print(F1)

In [59]:
# Set up a Suport Vector Machine Classifier
sv = LinearSVC(labelCol="label", featuresCol="features")

In [60]:
# Fit the model to the training data
svModel = sv.fit(train)

In [64]:
# Create predictions for the test data set
prediction = svModel.transform(test)

In [65]:
# Create predictions for the test data set
F1 = evaluator.evaluate(prediction)
print(F1)

0.6368192335910605


In [66]:
# Set up a Logistic Regression Classifier
lr = LogisticRegression(labelCol="label", featuresCol="features")

In [67]:
# Fit the model to the training data
lrModel = lr.fit(train)

In [68]:
# Create predictions for the test data set
prediction = lrModel.transform(test)

In [69]:
# Create predictions for the test data set
F1 = evaluator.evaluate(prediction)
print(F1)

0.6886713982540287


In [70]:
# Cross Validation setup for logistic regression
paramGrid = ParamGridBuilder().addGrid(lr.fitIntercept, [True, False]).addGrid(lr.standardization, [True, False]).build()

crossval = CrossValidator(estimator = lr, estimatorParamMaps = paramGrid, 
                          evaluator = MulticlassClassificationEvaluator(metricName="f1"))

In [71]:
# Train the cross validation model on the training data
cvModel = crossval.fit(train)

In [72]:
# Find average accuracy metrics for paramgrid results
cvModel.avgMetrics


[0.6420637251724449,
 0.6420637251724449,
 0.6558321231986786,
 0.6558321231986786]

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.